# TSP Problem

## 1. Defining a function to compute de distances between two gps coordinates.

In [1]:
from math import sin, cos, sqrt, atan2, radians

def distance(x, y):
    R = 6373.0
    
    lat1 = radians(selected_cities.loc[x,'lat'])
    lon1 = radians(selected_cities.loc[x,'lng'])
    lat2 = radians(selected_cities.loc[y,'lat'])
    lon2 = radians(selected_cities.loc[y,'lng'])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    
    return distance


# 2. Importing a dataframe that contains latitude and longitude coordinates of 15,493 cities from around the world.

In [2]:
import pandas as pd
cities_coordinates = pd.read_csv('worldcities.csv')

# 3. Selecting a group of 22 big cities

In [3]:
selected = ['Tokyo','New York','Mexico City','Rio de Janeiro','Los Angeles','Buenos Aires','Rome','Lisbon','Paris',
            'Munich','Changping','Delhi','Sydney','Moscow','Istanbul','Cape Town','Madrid','Seoul','London','Bangkok',
            'Toronto','Dubai']
selected_cities = cities_coordinates.loc[cities_coordinates['city'].isin(selected),['city','country','lat','lng']]
selected_cities.reset_index(inplace = True, drop = True)
selected_cities = selected_cities.loc[0:21,:]
selected_cities = selected_cities.drop('country', axis = 1)
selected_cities.set_index('city', inplace = True)

# 4. Computing the distances between each of them.

In [4]:
data = [[distance(i,j) for j in selected_cities.index] for i in selected_cities.index]

# 5. Running the GA Algorithm.

In [5]:
import numpy as np
import pandas as pd
import random

def initial(dv, pop_size):
    pop = [list(np.random.permutation(dv)) for i in range(pop_size)]
    return pop

def fitness_aux(x):
    y=[data[x[i]][x[i+1]] for i in range(len(x)) if i < (len(x)-1)]
    y.append(data[x[-1]][x[0]])
    return sum(y)

def fitness_function(x):
    y = [fitness_aux(x[i]) for i in range(len(x))]
    return y

def tournament_selection(x, size = 5):
    candidates = list(np.random.choice(range(len(x)), size))
    candidates_fitness = [fitness[i] for i in candidates]
    champion = candidates[candidates_fitness.index(min(candidates_fitness))]
    return champion

def select_parents(x):
    parents_index = [tournament_selection(x) for i in range(len(x))]
    parents = [x[i] for i in parents_index]
    return parents

def order_crossover(a,b):
    points = list(np.random.choice(range(1,len(a)), 2, replace=False))
    points.sort()
    point1, point2 = points[0], points[1]
    
    child_a = [-1]*len(a)
    child_a[point1 : point2] = a[point1 : point2]
    conflict_a = a[point1 : point2]
    mapping_a = [i for i in b[point2:] + b[:point2] if i not in conflict_a] 
    diff=len(a)-point2
    child_a[point2:] = mapping_a[:diff]
    child_a[:point1] = mapping_a[diff:]
    
    child_b = [-1]*len(b)
    child_b[point1 : point2] = b[point1 : point2]
    conflict_b = b[point1 : point2]
    mapping_b = [i for i in a[point2:] + a[:point2] if i not in conflict_b] 
    diff=len(b)-point2
    child_b[point2:] = mapping_b[:diff]
    child_b[:point1] = mapping_b[diff:]
    
    return child_a, child_b

def inversion_mutation(a):
    points = list(np.random.choice(range(1,len(a)), 2, replace=False))
    points.sort()
    point1, point2 = points[0], points[1]
    child = a[:point1] + a[point1:point2][::-1] + a[point2:]
    return child

def elitism_replacement(population, fitness, offspring, fitness_offspring):
    if min(fitness) < min(fitness_offspring):
        offspring[fitness_offspring.index(max(fitness_offspring))] = population[fitness.index(min(fitness))]
    return offspring

def save_best_fitness(population, fitness):
    best = fitness.index(min(fitness))
    return population[best], fitness[best]


decision_variables = list(range(len(data)))
population = initial(decision_variables, 20)
fitness = fitness_function(population)
best = save_best_fitness(population, fitness)
generation, best_fitness, fittest = [0], [best[1]], [str(best[0])]

for gen in range(1000):
    parents = select_parents(population)
    offspring = parents.copy()
    for i in range(0,len(population),2):
        if (np.random.uniform() < 0.6):
            offspring[i],offspring[i+1] = order_crossover(parents[i],parents[i+1])
    for i in range(len(population)):
        if (np.random.uniform() < 0.6):
            offspring[i] = inversion_mutation(offspring[i])
    fitness_offspring = fitness_function(offspring)
    population = elitism_replacement(population, fitness, offspring, fitness_offspring)
    fitness = fitness_function(population)
    best = save_best_fitness(population, fitness)
    generation.append(gen+1), best_fitness.append(best[1]), fittest.append(str(best[0]))

generation = pd.Series(generation)
best_fitness = pd.Series(best_fitness)
fittest = pd.Series(fittest)
run = pd.concat([generation, best_fitness, fittest], axis = 1)
run.columns = ['Generation', 'Fitness', 'Fittest']
run.drop_duplicates('Fittest', inplace=True)

# 6. Preparing the dataframe

In [6]:
def path(x):
    best_fitness_aux = run.loc[x,'Fittest'].replace(',','').replace('[','').replace(']','').split(' ')
    path_best_fitness = [int(i) for i in best_fitness_aux]
    path_best_fitness = path_best_fitness + [path_best_fitness[0]]
    return path_best_fitness

In [7]:
generation = lambda x: ['Generation_'+str(run.loc[x,'Generation'])]*len(path(x))
total_distance = lambda x: [run.loc[x,'Fitness']]*len(path(x))

In [8]:
all_path = []
all_generation = []
all_distances = []
for i in run.loc[:,'Generation']:
    all_path = all_path + path(i)
    all_generation = all_generation + generation(i) 
    all_distances = all_distances + total_distance(i)

In [9]:
all_generation = pd.Series(all_generation)
all_path = pd.Series(all_path)
all_distances = pd.Series(all_distances)

In [10]:
x_coordinate = [selected_cities.iloc[i,0] for i in all_path]
y_coordinate = [selected_cities.iloc[i,1] for i in all_path]
name_city = [selected_cities.index[i] for i in all_path]
x_coordinate = pd.Series(x_coordinate)
y_coordinate = pd.Series(y_coordinate)
name_city = pd.Series(name_city)

In [23]:
df = pd.concat([all_generation, all_path, all_distances, name_city, x_coordinate, y_coordinate], axis = 1)
df.columns = ['generation', 'city', 'distance', 'name_city', 'x_coordinate','y_coordinate']

In [24]:
df['norm_distance'] = ''

In [32]:
max_ = df['distance'].max()
min_ = df['distance'].min()
for idx in df.index:
    df.at[idx, 'norm_distance'] = (df['distance'].loc[idx] - min_)/(max_ - min_)

# 7. Plotting

In [34]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure(
    data=[go.Scattergeo(lat=df.loc[df.loc[:,"generation"] == 'Generation_0',"x_coordinate"] , 
                     lon=df.loc[df.loc[:,"generation"] == 'Generation_0',"y_coordinate"] ,
                     hoverinfo = 'text',
                     text = df.loc[df.loc[:,"generation"] == 'Generation_0',"name_city"],
                     mode="lines+markers",
                     line=dict(width=1, color="blue"),
                     marker=dict(size=4, color="red"))],
    layout=go.Layout(
#         xaxis=dict(range=[-60,60], autorange=False, zeroline=False),
#         yaxis=dict(range=[-150,160], autorange=False, zeroline=False),
        title_text="TSP Problem", hovermode="closest",
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None]),
                                   dict(label="Pause",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Scattergeo(lat=df.loc[df.loc[:,"generation"] == k,"x_coordinate"] , 
                     lon=df.loc[df.loc[:,"generation"] == k,"y_coordinate"] ,
                     text = df.loc[df.loc[:,"generation"] == k,"name_city"],
                     mode="lines+markers",
                     line=dict(width=((df.loc[df.loc[:,"generation"] == k,"norm_distance"].iloc[0])+0.1)*8, color="blue"),
                     marker=dict(size=4, color="red"))])

        for k in df.loc[:,"generation"].unique()]
)

fig.show()